sync linkedin contacts with google contacts?



In [ ]:
var importer = require('../Core');
var fs = require('fs');
var glob = require('glob');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';
var escapeFilename = (f) => {
    return f.replace(/[^a-z0-9-\s]/ig, '_')
}

var updateContact, listAllConnections, scrapeEntireLinkedInProfile,
    runSeleniumCell;
var syncLinkedInContacts = () => {
    var loaded;
    return importer.import([
        'update google contact',
        'selenium cell'
    ])
        .then(r => {
            updateContact = r[0];
            runSeleniumCell = r[1];
            return runSeleniumCell([
                'scrape linkedin contacts',
                'scrape entire linkedin profile'
            ]);
        })
        .then(r => {
            listAllConnections = r[0];
            scrapeEntireLinkedInProfile = r[1];
            loaded = glob.sync('**/linkedin-*.json', {cwd: project})
                .map(l => {
                    try {
                        return JSON.parse(fs.readFileSync(path.join(project, l)))[0].url.replace(
                            'linkedin.com',
                            '') + '/'
                    } catch (e) {
                        console.log(l);
                    }
                });
            return listAllConnections();
        })
        .then((connections) => {
            const filtered = connections.filter(c => loaded.indexOf(escapeFilename(c)) === -1 && loaded.indexOf(c) === -1);
            const percent = Math.round((connections.length - filtered.length) / connections.length * 100);
            console.log((connections.length - filtered.length) + ' / ' + connections.length + ' : '
                + percent
                + '%');
            console.log(filtered.slice(0, 30));
            return importer.runAllPromises(filtered.slice(0, 30).map(c => resolve => {
                var linkedIn;
                return scrapeEntireLinkedInProfile('https://linkedin.com' + c)
                    .then(r => {
                        linkedIn = r;
                        return updateContact({
                            displayName: r.name.split(/\s/).join('.*')
                        });
                    })
                    .then(r => {
                        console.log(linkedIn.name);
                        if (r.length === 0) {
                            r.push({});
                        }
                        Object.assign(r[0], linkedIn);
                        fs.writeFileSync(
                            project + '/linkedin-' + escapeFilename(linkedIn.name) + '.json',
                            JSON.stringify(r, null, 4));
                        return resolve(r);
                    })
                    .catch(e => {
                        console.log(e);
                        resolve()
                    })
            }))
        })
        .catch(e => console.log(e))
};

module.exports = syncLinkedInContacts;


scrape entire linkedin profile?



In [ ]:
var importer = require('../Core');var getContactInfo = (profile) => {    return client        .execute(url => {            var a = document.createElement('a');            a.setAttribute('href', url);            a.href = url;            a.style.position = 'absolute';            a.style.zIndex = 4294967295;            a.style.top = 0;            a.style.left = 0;            a.style.bottom = 0;            a.style.right = 0;            document.body.appendChild(a);            setTimeout(() => document.body.removeChild(a), 500)        }, profile)        .click('a[href*="' + profile + '"]')        .pause(2000)        .then(() => readLinkedInProfileInfo())        .catch(e => console.log(e))};var loadEntirePage = () => {    return client    // load all defered elements by scrolling        .then(() => getAllUntil(            false,            '//*[contains(@class, "pv-deferred-area")]/@id',            [],            (a, b) => a === b,            (i) => i < 30        ))        .pause(2000)        // show more all sections        .scroll(0, 0)        .then(() => importer.runAllPromises([            '//button[contains(., "See more")]',            '//button[contains(., "more skills")]',            '//button[contains(., "more education")]',            '//button[contains(., "more recommendations")]'        ].map(selector => resolve => {            return client                .isVisible(selector)                .then(is => is                    ? client.click(selector).pause(1000)                    : [])                .then(e => resolve())                .catch(e => {console.log; return resolve()})        })))        .catch(e => console.log(e))};var loginLinkedIn, readLinkedInProfileInfo, getAllXPath;var scrapeEntireLinkedInProfile = (profile) => {    var contact;
    return client        .then(() => importer.import([            'log in linkedin',            'scrape linkedin profile',        ], {client, getAllXPath, getCredentials}))        .then(r => {            loginLinkedIn = r[0];            readLinkedInProfileInfo = r[1];            return loginLinkedIn();        })        .then(() => getContactInfo(profile))        .then(r => {            contact = r;            return loadEntirePage();        })        // get extra profile info        .then(() => {            return getAllXPath({                summary: '//p[contains(@class, "section__summary")]//text()',                experience: [                    '//*[contains(@class, "profile-section__card")]|//*[contains(@class, "profile-section__section-info-item")]',                    ['.//text()']                ],                skills: [                    '//*[contains(@class, "skill-entity--featured")]|//*[contains(@class, "skill-entity__pill")]',                    ['.//text()']                ],                recommendations: [                    '//*[contains(@class, "recommendation-entity")]',                    ['.//text()']                ],                interests: [                    '//*[contains(@class, "interest-entity")]',                    ['.//text()']                ]            });        })        .then(r => {            return Object.assign(r, {                summary: r.summary.join(''),                experience: r.experience.map(e => typeof e[0] === 'string'                    ? e[0]                    : e[0].join('')                        .trim().split(/\s*\n+\s*/)),                skills: r.skills.map(e => typeof e[0] === 'string'                    ? e[0]                    : e[0].join('')                        .trim().split(/\s*\n+\s*/)),                recommendations: r.recommendations.map(e => typeof e[0] === 'string'                    ? e[0]                    : e[0].join('')                        .trim().split(/\s*\n+\s*/)),                interests: r.interests.map(e => typeof e[0] === 'string'                    ? e[0]                    : e[0].join('')                        .trim().split(/\s*\n+\s*/))            }, contact)        })        .catch(e => console.log(e))};module.exports = scrapeEntireLinkedInProfile;

scrape linkedin contacts?



In [ ]:
var importer = require('../Core');var fs = require('fs');var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;var project = PROFILE_PATH + '/Conversations';var getAllUntil, getAllXPath;var listAllConnections = () => {    var friends = [];    if (fs.existsSync(project + '/connections.json')) {        return Promise.resolve(JSON.parse(            fs.readFileSync(project + '/connections.json')));    }    return client        .then(() => importer.import([            'get all elements until',            'all xpath elements'        ], {client}))        .then(r => {            getAllUntil = r[0];            getAllXPath = r[1];            return client.getUrl();        })        .url('https://www.linkedin.com/mynetwork/invite-connect/connections/')        .pause(3000)        .then(() => getAllUntil(            false,            '//a[contains(@href, "/in/")]/@href',            friends,            (a, b) => a === b,            (i) => i < 100        ))        .then(r => r.filter((l, i, arr) => arr.indexOf(l) === i))        .then(r => {            fs.writeFileSync(                project + '/connections.json',                JSON.stringify(r, null, 4));            return r;        })        .catch(e => console.log(e))};module.exports = listAllConnections;

test linkedin scraper


In [1]:
var importer = require('../Core');$$.async();importer.import('sync linkedin contacts')    .then(syncLinkedInContacts => syncLinkedInContacts())    .then(r => $$.sendResult(r))    .catch(e => $$.sendError(e));

Initializing webdriver on localhost
[ '93252a702089791aa4f7698fa70f37d8' ]
999 / 2435 : 41%
Michael Peters
Gail Barnes, PMP® , CSM, MBA

Peter Piotr Góralczyk Goralczyk
Derek Stewart
Christian Hébert
🌟Jonathan Barry🌟
Chris Huie
Imelda Hartley
Ketan Patel
Jason Heath
Kristen Ramos
Luis Figueroa
Pankaj Likhmania
Chris Ronzio
Tyler Bloom
Braden Pollock
Anumit Jooloor
Scott Yacko
Ben Smith
Phil Potter
Jeff Coruccini, Employability Champion
Terri Kimble
Jordan Armstrong
Gary Bennett
Alexander Schwarz
Bill Doshier
Mary Juetten
James Goulka
Brett Benning


[ { resourceName: 'people/c5779287504560820322',
    etag: '%EgUBAgcJCxoNAQIDBAUGBwgJCgsMDSIMOE9lQWkrWkNjYms9',
    names: [ [Object] ],
    emailAddresses: [ [Object] ],
    experience: 
     [ [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       'Recommendations',
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       'People Also Viewed' ],
    interests: 
     [ [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array],
       [Array] ],
    recommendations: [],
    skil